![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ASSERTION.ipynb)

# `assertion` **Models**

This model extract entities using the NER model in the pipeline and assign assertion status for them with pretrained model. 

## 1. Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


## 2. Select the model and construct the pipeline

**Create the pipeline**

In [15]:
def run_pileline(word_embeddings, clinical_ner, clinical_assertion, sample_text):
  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")
  if word_embeddings == "biobert_pubmed_base_cased":

    word_embeddings = BertEmbeddings.pretrained(word_embeddings)\
        .setInputCols(["sentence",'token'])\
        .setOutputCol("embeddings")
  else:
    word_embeddings = WordEmbeddingsModel.pretrained(word_embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

  clinical_ner = MedicalNerModel.pretrained(clinical_ner, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

  ner_converter = NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

  clinical_assertion = AssertionDLModel.pretrained(clinical_assertion, "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

  nlpPipeline = Pipeline(stages=[
        documentAssembler, 
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        clinical_assertion
  ])

  #Create example inputs
  from pyspark.sql.types import StringType, IntegerType

  df = spark.createDataFrame(sample_text, StringType()).toDF('text')

  df.show(truncate = 100)

  #Use the pipeline to create outputs
  result = nlpPipeline.fit(df).transform(df)

  result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                      result.ner_chunk.begin, 
                                      result.ner_chunk.end,
                                      result.ner_chunk.metadata,
                                      result.assertion.result,
                                      result.assertion.metadata)).alias("cols"))\
          .select(F.expr("cols['0']").alias("chunk"),
                  F.expr("cols['1']").alias("begin"),
                  F.expr("cols['2']").alias("end"),
                  F.expr("cols['3']['entity']").alias("entity"),
                  F.expr("cols['4']").alias("assertion"),
                  F.expr("cols['5']['confidence']").alias("confidence")).show(truncate=30)

  #Visualize results

  from sparknlp_display import AssertionVisualizer

  viz = AssertionVisualizer()


  for j in range(len(sample_text)):
      viz.display(result = result.collect()[j], label_col = "ner_chunk", assertion_col = "assertion")
      print("\n\n")

## **assertion_dl_biobert**

In [5]:
## Generating Example Files ##
sample_text = [
    """ The patient is a 21-day-old Caucasian male here for 2 days of Congestion - mom has been Suctioning-Yellow-Discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but did not observe for any perioral cyanosis). Tactile temperature was detected in the patient the day before. And TYLENOL was given to the patient by his mother. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. He sleeps well, but has been more tired and has been fussy over the past 2 days. His parents has not observed any improvement during albuterol treatments in the ER. His urine-output problem decrease. Normally he has 8 to 10 wet diapers, now he has down to 4 wet diapers per 24 hours.""",
    """ The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basketball at the gym today. Two hours before the presentation, he fell and someone stepped on her ankle and unable to move leg and weakness. No other injuries were recorded. Adderall and Accutane has given.""",
    """ The patient is in with several medical problems. She has complains of numbness, tingling, and pain in the toes in her right foot described as moderate pain. She initially describes it as a sharp quality pain but is unable to characterize it more fully. She has had it for about a year but seems to be getting worsening. She has a few paraesthesias in the left toe as well and seems to involve all the toes of the right foot. They are not worse with walking. It seems to be worse when she is in bed. There is some radiation of the pain up her leg. She also continues to have bilateral shoulder pains without sinus allergies. She has hypothyroidism. She has thrombocythemia, insomnia, and hypertension.""",         
    """ The patient is a 5-month-old infant who presented on Monday. She has had a cold, cough, and runny nose for 2 days. Mom states she had no fever. Her appetite is good but she is drooling a lot. She had no difficulty breathing and her cough was described as dry and hacky. At that time, red right TM was observed on physical examination. Left TM was okay. She was fairly congested but looked happy and playful. She was started on Amoxil and Aldex and we told to recheck in 2 weeks to recheck her ear. She was having difficulty breathing. She was much more congested and she had a massive loss of appetite today. She also spiked a temperature yesterday of 102.6 and always having Secondary sleep disorders due to congestion.""",
    """ The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he has taken 3 nitroglycerin tablets sublingually in the last 1 hour and his pain is partially gone. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.""" ]

In [6]:
run_pileline("biobert_pubmed_base_cased", "ner_clinical_biobert", "assertion_dl_biobert", sample_text)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
ner_clinical_biobert download started this may take some time.
[OK!]
assertion_dl_biobert download started this may take some time.
[OK!]
+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
| The patient is a 21-day-old Caucasian male here for 2 days of Congestion - mom has been Suctioni...|
| The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basket...|
| The patient is in with several medical problems. She has complains of numbness, tingling, and pa...|
| The patient is a 5-month-old infant who presented on Monday. She has had a cold, cough, and runn...|
| The patient is a 40-year-ol

## **assertion_dl_healthcare**

In [7]:
## Generating Example Files ##
sample_text = [
    """ The patient is a 21-day-old Caucasian male here for 2 days of Congestion - mom has been Suctioning-Yellow-Discharge from the patient's nares, plus she has noticed some mild problems with his breathing during feeding time(but did not observe for any perioral cyanosis or retractions). Tactile temperature was detected in the patient the day before. TYLENOL was given to the patient by his mother. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. His parents has not observed any improvement during albuterol treatments in the ER. His urine-output problem decrease. Normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His bowel movement problems are yellow colored and soft in nature.""",
    """ The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basketball at the gym today. Two hours before the presentation, he fell and someone stepped on her ankle and unable leg movement problem and weakness. No other injuries were recorded. In the past, there has been no injury to the ankle. Adderall and Accutane has been given.""",
    """ The patient is in with several medical problems. She has complains of numbness, Tingle problem, and pain in the toes in her right foot described as moderate pain. She initially describes it as a sharp quality pain but is unable to characterize it more fully. She has had it for about a year but seems to be getting worsening. She has a few paraesthesias in the left toe as well and seems to involve all the toes of the right foot. They are not worse with walking. It seems to be worse when she is in bed. There is some radiation of the pain up her leg. She also continues to have bilateral shoulder pains without sinus allergies. She has hypothyroidism. She has thrombocythemia, insomnia, and hypertension.""",         
    """ The patient is a 5-month-old infant who presented on Monday. She has had a cold and Coughing and runny nose for 2 days. Mom states she had no fever. Her appetite is good but she is drooling a lot. She had no difficulty breathing and her cough was described as dry and hacky. At that time, red right TM was observedon physical examination. Left TM was okay. She was fairly congested but looked happy and playful. She was started on Amoxil and Aldex and we told to recheck in 2 weeks to recheck her ear. She was having difficulty breathing. She was much more congested and she had a massive loss of appetite today. She also spiked a temperature yesterday of 102.6 and always having Secondary sleep disorders due to congestion.""",
    """ The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath and diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he has taken 3 nitroglycerin tablets sublingually in the last 1 hour and his pain is partially gone. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.""" ]

In [8]:
run_pileline("embeddings_healthcare_100d", "ner_healthcare", "assertion_dl_healthcare", sample_text)

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]
ner_healthcare download started this may take some time.
[OK!]
assertion_dl_healthcare download started this may take some time.
[OK!]
+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
| The patient is a 21-day-old Caucasian male here for 2 days of Congestion - mom has been Suctioni...|
| The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basket...|
| The patient is in with several medical problems. She has complains of numbness, Tingle problem, ...|
| The patient is a 5-month-old infant who presented on Monday. She has had a cold and Coughing and...|
| The patient is a 40-year-old 

## **assertion_dl**

In [9]:
## Generating Example Files ##
sample_text = [
    """ The patient is a 21-day-old Caucasian male here for 2 days of Congestion - mom has been Suctioning-Yellow-Discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but did not observe for any perioral cyanosis or retractions). Tactile temperature was detected in the patient the day before. TYLENOL was given to the patient by his mother. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. His parents has not observed any improvement during albuterol treatments in the ER. His urine-output problem decrease. Normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His bowel movement problems are yellow colored and soft in nature.""",
    """ The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basketball at the gym today. Two hours before the presentation, he fell and someone stepped on her ankle and unable leg movement problem and weakness. No other injuries were recorded. In the past, there has been no injury to the ankle. Adderall and Accutane has given.""",
    """ The patient is in with several medical problems. She has complains of numbness, Tingle problem, and pain in the toes in her right foot described as moderate pain. She initially describes it as a sharp quality pain but is unable to characterize it more fully. She has had it for about a year but seems to be getting worsening. She has a few paraesthesias in the left toe as well and seems to involve all the toes of the right foot. They are not worse with walking. It seems to be worse when she is in bed. There is some radiation of the pain up her leg. She also continues to have bilateral shoulder pains without sinus allergies. She has hypothyroidism. She has thrombocythemia, insomnia, and hypertension.""",         
    """ The patient is a 5-month-old infant who presented on Monday. She has had a cold, Cough and runny nose for 2 days. Mom states she had no fever. Her appetite is good but she is drooling a lot. She had no difficulty breathing and her cough was described as dry and hacky. At that time, red right TM was observed on physical examination. Left TM was okay. She was fairly congested but looked happy and playful. She was started on Amoxil and Aldex and we told to recheck in 2 weeks to recheck her ear. She was having difficulty breathing. She was much more congested and she had a massive loss of appetite today. She also spiked a temperature yesterday of 102.6 and always having Secondary sleep disorders due to congestion.""",
    """ The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he has taken 3 nitroglycerin tablets sublingually in the last 1 hour and his pain is partially gone. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.""" ]

In [10]:
run_pileline("embeddings_clinical", "ner_clinical", "assertion_dl", sample_text)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
| The patient is a 21-day-old Caucasian male here for 2 days of Congestion - mom has been Suctioni...|
| The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basket...|
| The patient is in with several medical problems. She has complains of numbness, Tingle problem, ...|
| The patient is a 5-month-old infant who presented on Monday. She has had a cold, Cough and runny...|
| The patient is a 40-year-old white male who present

##**assertion_jsl**

In [11]:
## Generating Example Files ##
sample_text = [
    """ The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioning yellow discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but did not observe for any perioral cyanosis or retractions). Tactile temperature was detected in the patient the day before. And TYLENOL was given to the patient by his mother. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. He sleeps well, but has been more tired over the past 2 days. His parents has not observed any improvement during Albuterol inhalation in the ER. Normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His feces is yellow-colored and soft in nature.""",
    """ The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basketball at the gym today. Two hours before the presentation, he fell and someone stepped on her ankle and Unable leg movement problem and weakness. No other injuries were recorded. In the past, there has been no injury to the ankle. Adderall and Accutane medication has given.""",
    """ The patient is in with several medical problems. She has complains of numbness, tingling, and pain in the toes in her right foot described as moderate pain. She initially describes it as a sharp quality pain but is unable to characterize it more fully. She has had it for about a year but seems to be getting worsening. She has a few paraesthesias in the left toe as well and seems to involve all the toes of the right foot. They are not worse with walking. It seems to be worse when she is in bed. There is some radiation of the pain up her leg. She also continues to have bilateral shoulder pains without sinus allergies. She has hypothyroidism. She has thrombocythemia, insomnia, and hypertension.""",         
    """ The patient is a 5-month-old infant who presented on Monday. She has had a cold, cough, and runny nose for 2 days. Mom states she had no fever. Her appetite is good but she is drooling a lot. She had no difficulty breathing and her cough was described as dry and hacky. At that time, a right TM, which was RED on physical examination, was observed. Left TM was okay. She was fairly congested but looked happy and playful. She was started on Amoxil and Aldex and we told to recheck in 2 weeks to recheck her ear. She was having difficulty breathing. She was much more congested and she had a massive loss of appetite today. She also spiked a temperature yesterday of 102.6 and always having trouble sleeping secondary to congestion.""",
    """ The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he has taken 3 nitroglycerin tablets sublingually in the last 1 hour. Now his pain is gone. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.""",
    """ Mr. ABC is a 34-year-old man who presented to the emergency room with severe headache, blurred vision, nausea, 1 episode of vomiting and tinnitus after a Motor-Vehicle-Accident problem 5 hours ago. He was driving to the office in the morning when he was hit on the passenger side of the vehicle.  

    PAST HISTORY: Appendectomy 10 years ago. 

    FAMILY HISTORY: Allergy, Colon cancer, Diabetes, Heart disease, Hearing loss 

    SOCIAL HISTORY: Smoker.""",
    """ Miss M. is a 67-year-old lady, with past history of COPD and Hypertension, presents with a 3-weeks history of a lump in her right Breast. The lump appeared suddenly, also painful. 5 days ago, another lump has appeared in her right axilla. On examination, a 2 x 3 cm swelling was seen in the right Breast. It was firm and also non-tender and immobile. There was no discharge. Another 1x1 cm circumferential swelling was found in the right Axilla, which was freely mobile and also tender. Her family history is remarkable for Breast cancer (mother), cervical cancer (maternal grandmother), heart disease (father), COPD (Brother), dementia (Grandfather), diabetes (Grandfather), and CHF (Grandfather)."""  
            ]

In [12]:
run_pileline("embeddings_clinical", "ner_clinical", "assertion_jsl", sample_text)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_jsl download started this may take some time.
[OK!]
+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
| The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioni...|
| The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basket...|
| The patient is in with several medical problems. She has complains of numbness, tingling, and pa...|
| The patient is a 5-month-old infant who presented on Monday. She has had a cold, cough, and runn...|
| The patient is a 40-year-old white male who presen

##**assertion_jsl_large**

In [13]:
## Generating Example Files ##
sample_text = [
    """ The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioning yellow discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but did not observe for any perioral cyanosis or retractions). Tactile temperature was detected in the patient the day before. And TYLENOL was given to the patient by his mother. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. He sleeps well, but has been more tired over the past 2 days. His parents has not observed any improvement during Albuterol inhalation in the ER. Normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His feces is yellow-colored and soft in nature.""",
    """ The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basketball at the gym today. Two hours before the presentation, he fell and someone stepped on her ankle and Unable leg movement problem and weakness. No other injuries were recorded. In the past, there has been no injury to the ankle. Adderall and Accutane medication is given.""",
    """ The patient is in with several medical problems. She has complains of numbness, tingling, and pain in the toes in her right foot described as moderate pain. She initially describes it as a sharp quality pain but is unable to characterize it more fully. She has had it for about a year but seems to be getting worsening. She has a few paraesthesias in the left toe as well and seems to involve all the toes of the right foot. They are not worse with walking. It seems to be worse when she is in bed. There is some radiation of the pain up her leg. She also continues to have bilateral shoulder pains without sinus allergies. She has hypothyroidism. She has thrombocythemia, insomnia, and hypertension.""",         
    """ The patient is a 5-month-old infant who presented on Monday. She has had a cold, Cough, and runny nose for 2 days. Mom states she had no fever. Her appetite is good but she is drooling a lot. She had no difficulty breathing and her cough was described as dry and hacky. At that time, a right TM, which was RED on physical examination, was observed. Left TM was okay. She was fairly congested but looked happy and playful. She was started on Amoxil and Aldex and we told to recheck in 2 weeks to recheck her ear. She was having difficulty breathing. She was much more congested and she had a massive loss of appetite today. She also spiked a temperature yesterday of 102.6 and always having trouble sleeping secondary to congestion.""",
    """ The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admited prior episodes of similar pain before his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he has taken 3 nitroglycerin tablets sublingually in the last 1 hour. Now his pain is gone. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.""",
    """ Mr. ABC is a 34-year-old man who presented to the emergency room with severe headache, blurred vision, nausea, 1 episode of vomiting and tinnitus after a Motor-Vehicle-Accident problem 5 hours ago. He was driving to the office in the morning when he was hit on the passenger side of the vehicle.  

    PAST HISTORY: Appendectomy 10 years ago. 

    FAMILY HISTORY: Allergy, Colon cancer, Diabetes, Heart disease, Hearing loss 

    SOCIAL HISTORY: Smoker.""",
    """ Miss M. is a 67-year-old lady, with past history of COPD and Hypertension, presents with a 3-weeks history of a lump in her right Breast. The lump appeared suddenly, also painful. 5 days ago, another lump has appeared in her right axilla. On examination, a 2 x 3 cm swelling was seen in the right Breast. It was firm and also lose tender and immobile. There was no discharge. Another 1x1 cm circumferential swelling was found in the right Axilla, which was freely mobile and also tender. Her family history is remarkable for Breast cancer (mother), cervical cancer (maternal grandmother), heart disease (father), COPD (Brother), dementia (Grandfather), diabetes (Grandfather), and CHF (Grandfather)."""  
            ]

In [14]:
run_pileline("embeddings_clinical", "ner_clinical", "assertion_jsl_large", sample_text)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_jsl_large download started this may take some time.
[OK!]
+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
| The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioni...|
| The patient is a pleasent 17-year-old gentleman who suffered from an injury, when playing basket...|
| The patient is in with several medical problems. She has complains of numbness, tingling, and pa...|
| The patient is a 5-month-old infant who presented on Monday. She has had a cold, Cough, and runn...|
| The patient is a 40-year-old white male who 